# Pytorch로 딥러닝 제대로 배우기-중급
## Part2: 머신러닝 개론
### 딥러닝 파이프 라인

1. 입력 데이터
2. 모델
3. 최적화

#### (1) 입력 데이터

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 공개된 데이터 셋으로부터 학습 데이터 다운로드 
training_data = datasets.FashionMNIST(
    root="./dataset",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개된 데이터 셋으로부터 테스트 데이터 다운로드
test_data = datasets.FashionMNIST(
    root="./dataset",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|█████████████████████████████████████████████████████████████████| 26421880/26421880 [00:12<00:00, 2057792.36it/s]


Extracting ./dataset\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./dataset\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 108498.73it/s]


Extracting ./dataset\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./dataset\FashionMNIST\raw



100%|███████████████████████████████████████████████████████████████████| 4422102/4422102 [00:04<00:00, 1080534.23it/s]


Extracting ./dataset\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./dataset\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<?, ?it/s]

Extracting ./dataset\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./dataset\FashionMNIST\raw



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


#### (2) Creating Models

In [4]:
# CPU 또는 GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### (3) 최적화

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303643  [    0/60000]
loss: 2.296430  [ 6400/60000]
loss: 2.275220  [12800/60000]
loss: 2.270322  [19200/60000]
loss: 2.260694  [25600/60000]
loss: 2.227766  [32000/60000]
loss: 2.233229  [38400/60000]
loss: 2.194874  [44800/60000]
loss: 2.199667  [51200/60000]
loss: 2.173472  [57600/60000]
Test Error: 
 Accuracy: 47.4%, Avg loss: 2.168184 

Epoch 2
-------------------------------
loss: 2.173929  [    0/60000]
loss: 2.168318  [ 6400/60000]
loss: 2.111090  [12800/60000]
loss: 2.131733  [19200/60000]
loss: 2.080617  [25600/60000]
loss: 2.021435  [32000/60000]
loss: 2.047746  [38400/60000]
loss: 1.961887  [44800/60000]
loss: 1.974239  [51200/60000]
loss: 1.913769  [57600/60000]
Test Error: 
 Accuracy: 56.8%, Avg loss: 1.906193 

Epoch 3
-------------------------------
loss: 1.930078  [    0/60000]
loss: 1.907859  [ 6400/60000]
loss: 1.788549  [12800/60000]
loss: 1.839540  [19200/60000]
loss: 1.721729  [25600/60000]
loss: 1.669840  [32000/600

#### (4) 모델 저장

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


#### (4) 모델 호출

In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
